In [1]:
# ==============================================================================
# CELLA 0: SETUP TOTALE (MINIMAL & STABILE)
# ==============================================================================
import sys
import os
from IPython.display import clear_output

# 1. BLOCCO MODULI PROBLEMATICI
sys.modules["vllm"] = None
sys.modules["vllm.sampling_params"] = None

print("⏳ Setup Ambiente in corso... (Attendere, output nascosto)")

# 2. INSTALLAZIONE & AGGIORNAMENTO SILENZIOSO
# Scarica l'ultima versione di Unsloth da Git e aggiorna automaticamente 
# PyTorch e Transformers alle versioni più recenti e compatibili.
!pip install --upgrade --no-cache-dir --quiet \
    "torch" "torchvision" "torchaudio" \
    "transformers" "trl" "peft" "accelerate" "bitsandbytes" \
    "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" \
    "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git" \
    "pillow" "scikit-learn" "pandas"

# 3. VERIFICA E PULIZIA
clear_output()

import torch
import unsloth
import transformers
from PIL import Image

print(f"✅ Ambiente Pronto e Pulito.")
print(f"   • GPU: {torch.cuda.get_device_name(0)}")
print(f"   • PyTorch: {torch.__version__}")
print(f"   • Unsloth: {unsloth.__version__}")
print(f"   • Transformers: {transformers.__version__}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/conda/lib/python3.11/site-packages/pandas/core/computation/expressions.py:22: UserWarning: Pandas requires version '2.10.2' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Unsloth: Using MoE backend 'grouped_mm'
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Ambiente Pronto e Pulito.
   • GPU: Tesla V100S-PCIE-32GB
   • PyTorch: 2.10.0+cu128
   • Unsloth: 2026.2.1
   • Transformers: 4.57.6


In [2]:
import torch
import os
import gc
import json
import shutil
import time
import random
import numpy as np
import unsloth
from datetime import datetime
from datasets import load_from_disk
from trl import SFTTrainer, SFTConfig
from unsloth import FastVisionModel, UnslothVisionDataCollator, is_bfloat16_supported
from transformers import TrainerCallback, set_seed

# ==============================================================================
# 1. CONFIGURAZIONE GLOBALE (Fissa per tutte le run)
# ==============================================================================
SEEDS = [101, 285, 3692, 92]  # <--- LISTA DI SEED DA TESTARE
NUM_EPOCHS = 5                # <--- NUMERO DI EPOCHE PER OGNI RUN (con SEED X)
MODEL_ID = "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit"
MODEL_SHORTNAME = "Qwen2.5-VL-M1-Detection"
DATASET_PATH = os.path.join("DATASET_ITA", "PROCESSED_DATA", "HF_DATASETS", "M1_detection")

# SYSTEM PROMPT
SYSTEM_INSTRUCTION_M1 = """Sei un classificatore binario esperto per la moderazione di contenuti social.
Valuta CONGIUNTAMENTE il commento testuale e i frame del video associato.

Il tuo compito è stabilire OGGETTIVAMENTE se è presente contenuto offensivo riconducibile ad una di queste categorie:
- Flaming: insulti, linguaggio aggressivo, minacce, parolacce dirette a una persona o gruppo.
- Denigration: umiliazione, disprezzo, svalutazione o ridicolizzazione di una persona o gruppo.
- Sexual: molestie, allusioni o offese a sfondo sessuale, oggettivazione o inviti espliciti.
- Racism: stereotipi, offese o discriminazione verso etnia, razza, cultura o nazionalità.

Regola decisionale:
- Rispondi 1 se rilevi contenuto offensivo riconducibile alle categorie sopra indicate.
- Rispondi 0 se il contenuto è neutro, positivo, o se si tratta di critica costruttiva/ironia non offensiva.

Linee guida:
- Basati solo sull'evidenza presente nei dati (testo + immagini).
- Non essere né troppo severo né troppo permissivo: attieniti alle definizioni.

Formato di output (OBBLIGATORIO):
Rispondi esclusivamente con un singolo carattere: 1 oppure 0.
Non aggiungere spiegazioni, punteggiatura o altro testo."""

# Callback per monitoraggio
class RealTimePrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"📝 Step: {state.global_step:4d} | Epoch: {logs['epoch']:.2f} | Loss: {logs['loss']:.4f}")

# ==============================================================================
# 2. CARICAMENTO E FORMATTAZIONE DATASET (Una volta sola per efficienza)
# ==============================================================================
print("📂 Caricamento Dataset HF (Eseguito una volta sola)...")
dataset_raw = load_from_disk(DATASET_PATH)

def has_valid_images(sample):
    user_msg = sample["messages"][0]
    for item in user_msg["content"]:
        if item["type"] == "image":
            raw_path = item["image"]
            clean_path = raw_path.replace("file://", "")
            check_path = "/" + clean_path.lstrip("/") if clean_path else ""
            if not os.path.exists(check_path):
                return False
    return True

# Filtriamo eventuali immagini rotte
train_valid = dataset_raw["train"].filter(has_valid_images, desc="Filter Valid Imgs")
val_valid = dataset_raw["val"].filter(has_valid_images, desc="Filter Valid Imgs")

def format_multimodal_sample(sample):
    raw_user_msg = sample["messages"][0]
    raw_assistant_msg = sample["messages"][1] 
    user_content = []
    
    for item in raw_user_msg["content"]:
        if item["type"] == "image":
            raw_path = item["image"]
            clean_path = raw_path.replace("file://", "")
            clean_path = "/" + clean_path.lstrip("/") 
            final_path = f"file://{clean_path}"
            user_content.append({"type": "image", "image": final_path})
        elif item["type"] == "text":
            text_clean = item["text"].replace("Commento:", "").strip().strip('"').strip("'")
            text_final = f"Commento: \"{text_clean}\""
            user_content.append({"type": "text", "text": text_final})
            
    label_text = raw_assistant_msg["content"][0]["text"]

    new_messages = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_INSTRUCTION_M1}]},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": [{"type": "text", "text": label_text}]}
    ]
    return {"messages": new_messages}

print("🔄 Formattazione Dataset...")
train_dataset = train_valid.map(format_multimodal_sample, batched=False, desc="Formatting Train")
val_dataset = val_valid.map(format_multimodal_sample, batched=False, desc="Formatting Val")
print(f"✅ Dataset Caricato e Formattato. Train: {len(train_dataset)} | Val: {len(val_dataset)}")


# ==============================================================================
# 3. MEGA-LOOP DI TRAINING (TUTTI I SEEDS)
# ==============================================================================
print(f"\n🚀 AVVIO SESSIONE DI TRAINING SU {len(SEEDS)} SEED: {SEEDS}")

for seed_idx, TRAINING_SEED in enumerate(SEEDS):
    print("\n" + "#"*60)
    print(f"🎬 RUN {seed_idx + 1}/{len(SEEDS)} | SEED CORRENTE: {TRAINING_SEED}")
    print("#"*60)

    # --- FIX DETERMINISMO GLOBALE ---
    print(f"🔒 Fissaggio Seed Globali a {TRAINING_SEED}...")
    random.seed(TRAINING_SEED)
    np.random.seed(TRAINING_SEED)
    torch.manual_seed(TRAINING_SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(TRAINING_SEED)
    set_seed(TRAINING_SEED)
    # -------------------------------------------------------

    # Definizione Output Directory Dinamica
    OUTPUT_DIR = f"outputs/{MODEL_SHORTNAME}_Seed_{TRAINING_SEED}"
    print(f"📂 Cartella Output Run: {OUTPUT_DIR}")

    # --- A. CARICAMENTO MODELLO ---
    print(f"⏳ Inizializzazione Modello (Seed {TRAINING_SEED})...")
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = MODEL_ID,
        load_in_4bit = True,
        use_gradient_checkpointing = "unsloth",
    )

    model = FastVisionModel.get_peft_model(
        model,
        r = 16,
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha = 16,
        lora_dropout = 0,
        bias = "none",
        random_state = TRAINING_SEED,
        use_rslora = False,
        loftq_config = None,
    )
    FastVisionModel.for_training(model)

    # --- B. CONFIGURAZIONE TRAINER ---
    training_args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        num_train_epochs = NUM_EPOCHS,
        learning_rate = 5e-5,
        lr_scheduler_type = "cosine",
        warmup_ratio = 0.1,
        weight_decay = 0.01,
        optim = "adamw_8bit",
        max_grad_norm = 0.3,
        
        # Salvataggio
        eval_strategy = "epoch",
        save_strategy = "epoch",
        save_total_limit = None,
        load_best_model_at_end = False,
        metric_for_best_model = "eval_loss",
        greater_is_better = False,
        
        # Hardware & Path
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        gradient_checkpointing = True,
        logging_steps = 10,
        output_dir = OUTPUT_DIR,
        report_to = "none",
        
        # Unsloth
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        seed = TRAINING_SEED,
    )

    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        data_collator = UnslothVisionDataCollator(model, tokenizer),
        train_dataset = train_dataset,
        eval_dataset = val_dataset,
        args = training_args,
        callbacks = [RealTimePrinterCallback()],
    )

    # --- C. ESECUZIONE TRAINING ---
    print(f"🔥 Avvio Training Seed {TRAINING_SEED}...")
    torch.cuda.empty_cache()
    
    start_time = time.time()
    trainer_stats = trainer.train()
    end_time = time.time()
    
    total_duration = (end_time - start_time) / 60
    final_train_loss = trainer_stats.training_loss
    global_steps_done = trainer_stats.global_step

    print(f"✅ Training Finito. Durata: {total_duration:.2f} min | Loss: {final_train_loss:.4f}")

    # --- D. SALVATAGGIO ---
    ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter_latest")
    REPORT_FILENAME = f"training_report_Seed_{TRAINING_SEED}.json"
    REPORT_PATH = os.path.join(OUTPUT_DIR, REPORT_FILENAME)
    ZIP_FILENAME = f"{MODEL_SHORTNAME}_Seed_{TRAINING_SEED}_FULL_CHECKPOINTS"
    
    # Cartella Padre per lo ZIP (Per evitare Inception bug)
    PARENT_DIR = os.path.dirname(OUTPUT_DIR)
    ZIP_FULL_PATH = os.path.join(PARENT_DIR, ZIP_FILENAME)

    os.makedirs(ADAPTER_PATH, exist_ok=True)
    
    print(f"💾 Salvataggio Artifacts...")
    model.save_pretrained(ADAPTER_PATH)
    tokenizer.save_pretrained(ADAPTER_PATH)

    # Report JSON
    peft_config_data = "N/A"
    try:
        raw_config = getattr(model, "peft_config", None)
        if isinstance(raw_config, dict) and raw_config.get("default"):
            peft_config_data = str(raw_config["default"])
    except: pass

    full_report = {
        "1_META_INFO": {
            "timestamp_end": datetime.now().isoformat(),
            "model_shortname": MODEL_SHORTNAME,
            "seed": TRAINING_SEED,
            "task": "M1 Detection - Training Loop"
        },
        "4_TRAINING_PERFORMANCE": {
            "total_duration_minutes": total_duration,
            "final_training_loss": final_train_loss,
            "global_steps": global_steps_done,
            "epochs": training_args.num_train_epochs
        },
        "5_LORA_PARAMS": peft_config_data,
        "7_ARTIFACTS": {
            "checkpoints_location": "Inside ZIP archive",
            "zip_path": f"{ZIP_FULL_PATH}.zip"
        },
        "8_FULL_LOG_HISTORY": getattr(trainer.state, "log_history", [])
    }

    with open(REPORT_PATH, "w", encoding="utf-8") as f:
        json.dump(full_report, f, indent=4, ensure_ascii=False)

    print(f"📦 Compressione ZIP in corso (attendere)...")
    shutil.make_archive(
        base_name=ZIP_FULL_PATH, 
        format='zip', 
        root_dir=OUTPUT_DIR
    )
    print(f"   -> ZIP creato: {ZIP_FULL_PATH}.zip")

    # --- E. PULIZIA MEMORIA ---
    print(f"🧹 Pulizia VRAM per il prossimo seed...")
    try:
        del model
        del trainer
        del tokenizer
    except: pass
    
    gc.collect()
    torch.cuda.empty_cache()
    print("✨ Ambiente pulito.\n")

print("\n🎉 TUTTE LE RUN PER M1 SONO COMPLETATE CON SUCCESSO!")

📂 Caricamento Dataset HF (Eseguito una volta sola)...


Filter Valid Imgs:   0%|          | 0/2373 [00:00<?, ? examples/s]

Filter Valid Imgs:   0%|          | 0/710 [00:00<?, ? examples/s]

🔄 Formattazione Dataset...


Formatting Train:   0%|          | 0/2373 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,373 | Num Epochs = 5 | Total steps = 745
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 47,589,376 of 8,339,756,032 (0.57% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Epoch,Training Loss,Validation Loss


📝 Step:   10 | Epoch: 0.07 | Loss: 4.8030
📝 Step:   20 | Epoch: 0.13 | Loss: 4.1080
📝 Step:   30 | Epoch: 0.20 | Loss: 2.9080
📝 Step:   40 | Epoch: 0.27 | Loss: 2.1999
📝 Step:   50 | Epoch: 0.34 | Loss: 1.7117
📝 Step:   60 | Epoch: 0.40 | Loss: 1.1156
📝 Step:   70 | Epoch: 0.47 | Loss: 0.4184
📝 Step:   80 | Epoch: 0.54 | Loss: 0.1795
📝 Step:   90 | Epoch: 0.61 | Loss: 0.1531
📝 Step:  100 | Epoch: 0.67 | Loss: 0.1431
📝 Step:  110 | Epoch: 0.74 | Loss: 0.1389


KeyboardInterrupt: 